In [1]:
import os
import nni
import yaml
import argparse
import GPUtil
import numpy as np
import pickle
import dateutil
import datetime
import pandas as pd
from UCTB.preprocess.GraphGenerator import GraphGenerator
from UCTB.dataset import NodeTrafficLoader

Using TensorFlow backend.


In [5]:
with open("E:\\UCTB_Dataset\\1h\\Bike_NYC.pkl","rb") as fp:
    data_loader = pickle.load(fp)

In [6]:
data_loader

{'TimeRange': ['2013-07-01', '2017-09-30'],
 'TimeFitness': 60,
 'Node': {'TrafficNode': array([[1, 1, 2, ..., 0, 0, 0],
         [1, 1, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [3, 0, 6, ..., 0, 0, 0],
         [3, 0, 9, ..., 0, 0, 0],
         [1, 0, 7, ..., 0, 0, 0]]),
  'TrafficMonthlyInteraction': array([[[ 67,   2,   8, ...,   0,   0,   0],
          [  2,  70,  11, ...,   0,   0,   0],
          [  8,  19, 127, ...,   0,   0,   0],
          ...,
          [  0,   0,   0, ...,   0,   0,   0],
          [  0,   0,   0, ...,   0,   0,   0],
          [  0,   0,   0, ...,   0,   0,   0]],
  
         [[ 43,   4,  11, ...,   0,   0,   0],
          [  2,  85,  12, ...,   0,   0,   0],
          [ 11,  23, 193, ...,   0,   0,   0],
          ...,
          [  0,   0,   0, ...,   0,   0,   0],
          [  0,   0,   0, ...,   0,   0,   0],
          [  0,   0,   0, ...,   0,   0,   0]],
  
         [[ 38,  10,  11, ...,   0,   0,   0],
          [  3,

In [7]:
traffic = data_loader['Node']['TrafficNode']

In [8]:
traffic.shape

(37248, 820)

# Generate demand_stdata.csv

In [47]:
#stationInfo = data_loader['Node']['StationInfo']
col = ["S_{}".format(x) for x in range(traffic.shape[1])]

In [28]:
start = dateutil.parser.parse(data_loader['TimeRange'][0])
fitness = data_loader['TimeFitness'] // 60

ind = [(start+ datetime.timedelta(hours=fitness*x)).strftime("%Y-%m-%d %H:%M:%S") for x in range(traffic.shape[0])]

In [30]:
df = pd.DataFrame(traffic,columns=col,index=ind)

In [31]:
df.to_csv("demand_stdata.csv")

In [15]:
dateutil.parser.parse("2013-07-01").timestamp()

1372608000.0

In [ ]:
2013/7/1  1:49:00

# spatial_node

In [51]:
stationInfo = data_loader['Node']['StationInfo']

In [52]:
stationInfo

[['116', '2013-07-01 00:31:49', 40.74177603, -74.00149746, 'W 17 St & 8 Ave'],
 ['119',
  '2013-07-02 07:07:40',
  40.69608941,
  -73.97803415,
  'Park Ave & St Edwards St'],
 ['120',
  '2013-07-01 06:01:48',
  40.68676793,
  -73.95928168,
  'Lexington Ave & Classon Ave'],
 ['127',
  '2013-07-01 00:17:09',
  40.73172428,
  -74.00674436,
  'Barrow St & Hudson St'],
 ['128',
  '2013-07-01 00:25:02',
  40.72710258,
  -74.00297088,
  'MacDougal St & Prince St'],
 ['137',
  '2013-07-01 07:22:05',
  40.761628,
  -73.972924,
  'E 56 St & Madison Ave'],
 ['143',
  '2013-07-01 00:44:46',
  40.69239502,
  -73.99337909,
  'Clinton St & Joralemon St'],
 ['144',
  '2013-07-01 07:16:39',
  40.69839895,
  -73.98068914,
  'Nassau St & Navy St'],
 ['146',
  '2013-07-01 00:02:23',
  40.71625008,
  -74.0091059,
  'Hudson St & Reade St'],
 ['147',
  '2013-07-01 00:56:34',
  40.71542197,
  -74.01121978,
  'Greenwich St & Warren St'],
 ['150', '2013-07-01 02:55:09', 40.7208736, -73.98085795, 'E 2 St & Avenu

In [67]:
sta_ind = ["S_{}".format(x) for x in range(len(stationInfo))]

In [68]:
station_matrix = [stationInfo[x][2:5] for x in range(len(stationInfo))]

In [69]:
col = ["latitude","longitude","remark"]

In [70]:
sta_df = pd.DataFrame(station_matrix,index=sta_ind,columns=col)

In [72]:
sta_df.to_csv("spatial_node.csv")

# Graph

In [2]:
loader = NodeTrafficLoader(dataset="Bike",city="NYC")

traffic (37248, 717)
test: (3725, 717)
train: (33523, 717)
test: (4397, 717)


In [3]:
graph = "distance-interaction-correlation"
graph_obj = GraphGenerator(graph=graph,
                           data_loader=loader,
                           threshold_distance=1000,
                           threshold_correlation=0,
                           threshold_interaction=500)

In [9]:
for ind,curr in enumerate(graph.split("-")):
    df_G = staticGraphV2(graph_obj.AM[ind])
    df_G.to_csv("{}_graph.csv".format(curr))

In [82]:
graph_obj.AM.shape

(3, 717, 717)

In [83]:
demo = graph_obj.AM[0]

In [84]:
demo.shape

(717, 717)

In [86]:
templete = "S{}_TO_S{}"

In [4]:
def staticGraph(G):
    templete = "S{}_TO_S{}"
    col = []
    element = []
    for i in range(G.shape[0]):
        for j in range(G.shape[1]):
            col.append(templete.format(i,j))
            element.append(G[i][j])
    element = np.reshape(np.array(element),[1,-1])
    return pd.DataFrame(element,columns=col,index=["static"])

In [6]:
def staticGraphV2(G):
    template = "S_{}"
    col = ["start","end","weight"]
    element = []
    for i in range(G.shape[0]):
        for j in range(G.shape[1]):
            if G[i][j] > 0:
                element.append([template.format(i),template.format(j),G[i][j]])
    element = np.reshape(np.array(element),[-1,3])
    return pd.DataFrame(element,columns=col,index=["static"]*element.shape[0])

In [92]:
df_G = staticGraph(graph_obj.AM[0])

In [96]:
curr = "distance"

In [97]:
df_G.to_csv("{}_network.csv".format(curr))

In [95]:
np.where(df_G > 0)

(array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 array([    22,     41,     52, ..., 513636, 513668, 513900], dtype=int64))

# tmp

In [1]:
import numpy as np
import pandas as pd
import os


data_path = "/Users/chenliyue/Documents/GitHub/Urban-Dataset/Bike_NYC/Graph"

In [2]:
for path in os.listdir(data_path):
    print(path)
    if path.endswith("csv"):
        data = pd.read_csv(os.path.join(data_path,path))
        data = data[['start','end','weight']]
        data.to_csv(path,index=False)


distance_graph.csv
correlation_graph.csv
.DS_Store
interaction_graph.csv
